In [1]:
%load_ext autoreload
%autoreload 2

import kyklos as ky
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'browser'
import heyoka as hy

In [2]:
# build EOMs for basic 2-body system
x, y, z, vx, vy, vz = hy.make_vars("x", "y", "z", "vx", "vy", "vz")
r = hy.sqrt(x**2 + y**2 + z**2)
mu=3.986004418e5
# Assemble system
sys = [
    (x, vx),
    (y, vy),
    (z, vz),
    (vx, -mu * x / r**3),
    (vy, -mu * y / r**3),
    (vz, -mu * z / r**3)
]
state_kep = ky.OE([7500,0,0,0,0,0],'kep',mu=mu)
state_cart = ky.OE.to_cartesian(state_kep)

t_end = 2*np.pi*np.sqrt(7500**3/mu)
grid = np.linspace(0,t_end,1000)
ta = hy.taylor_adaptive(sys,[0.0] * 6)
ta

C++ datatype            : double
Tolerance               : 2.220446049250313e-16
High accuracy           : false
Compact mode            : false
Taylor order            : 20
Dimension               : 6
Time                    : 0
State                   : [0, 0, 0, 0, 0, 0]

In [ ]:
ta.state[:] = state_cart.elements
traj = ta.propagate_grid(grid)[5]
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=traj[:,0], 
    y=traj[:,1],
    mode='lines',
    name='Trajectory',
    line=dict(color='blue', width=2)
))
fig.update_yaxes(scaleanchor="x", scaleratio=1)  # Equal aspect ratio

fig.show()

In [ ]:
ta.time = 0.0
ta.state[:] = state_cart.elements
c_out = ta.propagate_until(t_end, c_output=True)[4]
traj_c = c_out(grid)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=traj_c[:,0], 
    y=traj_c[:,1],
    mode='lines',
    name='Trajectory',
    line=dict(color='red', width=2)
))
fig.update_yaxes(scaleanchor="x", scaleratio=1)  # Equal aspect ratio

fig.show()

In [5]:
nan_state = np.array([7000, 0, 0, np.nan, 0, 7.5])
inf_state = np.array([7000, np.inf, 0, 0, 0, 7.5])
zero_state = np.array([0, 0, 0, 1, 1, 1])
ta.time = 0.0
ta.state[:] = zero_state
c_out = ta.propagate_until(t_end, c_output=True)[4]
ta, c_out
# traj_c = c_out(grid)
# c_out

(C++ datatype            : double
 Tolerance               : 2.220446049250313e-16
 High accuracy           : false
 Compact mode            : false
 Taylor order            : 20
 Dimension               : 6
 Time                    : nan
 State                   : [nan, nan, nan, nan, nan, nan],
 None)

In [8]:
ta.time = 0.0
ta.state[:] = state_cart.elements
c_out = ta.propagate_until(t_end, c_output=True)[4]
traj_c = c_out(grid)
c_out(t_end)

array([ 7.50000000e+03,  3.76341940e-11,  0.00000000e+00, -3.67642326e-14,
        7.29018008e+00,  0.00000000e+00])

In [7]:
sys = ky.System('2body', ky.EARTH)
orbit = ky.OE(a=7000, e=0.01, i=0, omega=0, w=0, nu=0)
        
traj2 = sys.propagate(orbit, t_start=100, t_end=0)
traj1 = sys.propagate(orbit, t_start=0, t_end=100)
grid1 = traj1.sample_raw()
print(f"traj1 t0 = {traj1.t0}, tf = {traj1.tf}")
print(f"traj1 times array is {np.linspace(traj1.t0, traj1.tf, 100)}")
grid2 = traj2.sample_raw()
print(f"traj2 t0 = {traj2.t0}, tf = {traj2.tf}")
print(f"traj2 times array is {np.linspace(traj2.t0, traj2.tf, 100)}")

Compiling 2body integrator...
âœ“ Compilation complete
traj1 t0 = 0.0, tf = 100.0
traj1 times array is [  0.           1.01010101   2.02020202   3.03030303   4.04040404
   5.05050505   6.06060606   7.07070707   8.08080808   9.09090909
  10.1010101   11.11111111  12.12121212  13.13131313  14.14141414
  15.15151515  16.16161616  17.17171717  18.18181818  19.19191919
  20.2020202   21.21212121  22.22222222  23.23232323  24.24242424
  25.25252525  26.26262626  27.27272727  28.28282828  29.29292929
  30.3030303   31.31313131  32.32323232  33.33333333  34.34343434
  35.35353535  36.36363636  37.37373737  38.38383838  39.39393939
  40.4040404   41.41414141  42.42424242  43.43434343  44.44444444
  45.45454545  46.46464646  47.47474747  48.48484848  49.49494949
  50.50505051  51.51515152  52.52525253  53.53535354  54.54545455
  55.55555556  56.56565657  57.57575758  58.58585859  59.5959596
  60.60606061  61.61616162  62.62626263  63.63636364  64.64646465
  65.65656566  66.66666667  67.67676768 

In [8]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=grid1[:,0], 
    y=grid1[:,1],
    mode='lines',
    name='Trajectory',
    line=dict(color='red', width=2)
))
fig.add_trace(go.Scatter(
    x=grid2[:,0], 
    y=grid2[:,1],
    mode='lines',
    name='Trajectory',
    line=dict(color='blue', width=2)
))
fig.update_yaxes(scaleanchor="x", scaleratio=1)  # Equal aspect ratio

fig.show()

In [ ]:
from kyklos import earth_drag, Satellite, OrbitalElements
import plotly.io as pio
pio.renderers.default = 'browser'

# Create Earth system with J2 and drag
sys = earth_drag()

# Define satellite properties
sat = Satellite(
    mass=500,              # kg
    drag_coeff=2.2,        # Dimensionless
    cross_section=10,      # m²
    inertia=np.eye(3)*100  # kg⋅m²
)

# Low Earth orbit subject to drag
orbit = OrbitalElements(a=6578, e=0.001, i=0.9, 
			omega=0, w=0, nu=0, system=sys)

# Propagate with satellite parameters
traj = sys.propagate(
    orbit,
    t_start=0,
    t_end=86400,  # 1 day
    satellite_params={'Cd_A': sat.drag_coeff * sat.cross_section, 
                      'mass': sat.mass}
)

# Observe altitude decay
times = np.linspace(0, 86400, 1000)
states = traj.evaluate(times, element_type='kep')
altitudes = [s.a - 6378.137 for s in states]

import plotly.graph_objects as go
fig = go.Figure(data=go.Scatter(x=times/3600, y=altitudes))
fig.update_layout(xaxis_title='Time [hours]', yaxis_title='Altitude [km]')
fig.show()
